**Bag of Words** -  order of words is ignored and word counts collected.
1.  Extract salient features from each post and store it as a vector per post
2.  Compute clustering on the vectors
3.  From this cluster, fetch a handful of posts having a similarity to the post in question. (diversity)

#Preprocessing
###Converting raw text into a bag of words

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [2]:
content = ["How to format my hard disk", "Hard disk format problems"]
X = vectorizer.fit_transform(content)
vectorizer.get_feature_names()

['disk', 'format', 'hard', 'how', 'my', 'problems', 'to']

In [3]:
print(X.toarray().transpose())

[[1 1]
 [1 1]
 [1 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]]


##Counting Words

In [113]:
import pandas as pd
def read_directory_to_list(directory='./repo/ch03/data/toy'):
    import os
    result = pd.DataFrame(columns=['Post Filename','Post Content'])
    for fn in os.listdir(directory):
        filename = directory+'/'+fn
        with open(filename, 'r') as f:
            result = result.append({'Post Filename': fn,
                                    'Post Content': f.read()},
                          ignore_index=True)
    return result
posts = read_directory_to_list()
posts

,Post Filename,Post Content
0,01.txt,This is a toy post about machine learning. Act...
1,02.txt,Imaging databases provide storage capabilities.
2,03.txt,Most imaging databases save images permanently.\n
3,04.txt,Imaging databases store data.
4,05.txt,Imaging databases store data. Imaging database...


In [114]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1)

#We have to notify the vectorizer about the full dataset up front
#so that it knows what words are to be expected.
posts = posts['Post Content']
X_train = vectorizer.fit_transform(posts)
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d"%(num_samples, num_features))

#samples: 5, #features: 25


In [48]:
#We can now vectorize our new post.
new_post = "imaging databases"
new_post_vec = vectorizer.transform([new_post]) #note [list]

#returns a sparse matrix
print("Returns sparse array")
print(new_post_vec)
print()

new_post_vec = new_post_vec.toarray()
print('to array:')
print(new_post_vec)

Returns sparse array
  (0, 5)	1
  (0, 7)	1

to array:
[[0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


###Naive Version - Euclidean distance


In [115]:
import scipy as sp
def dist_raw(v1, v2):
    delta = v1-v2
    return sp.linalg.norm(delta) #calculates euclidean difference

def find_euclidean_closest(new_post, dist_func, X_train=X_train):
    print("New post: %s"%(new_post))
    print()
    new_post_vec = vectorizer.transform([new_post])
    new_post_vec = new_post_vec.toarray()
    
    best_doc = None
    best_dist = sp.inf
    best_i = None
    for i, post in enumerate(posts):
        if post == new_post:
            continue
        post_vec = X_train.getrow(i).toarray()
        d = dist_func(post_vec, new_post_vec)
        print("=== Post %i with dist=%.2f: %s"%(i,d,post))
        if d<best_dist:
            best_dist = d
            best_i = i

    print("Best post is %i with dist=%.2f"%(best_i, best_dist))
    
find_euclidean_closest("imaging databases", dist_raw)

New post: imaging databases

=== Post 0 with dist=4.00: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=1.73: Imaging databases provide storage capabilities.
=== Post 2 with dist=2.00: Most imaging databases save images permanently.

=== Post 3 with dist=1.41: Imaging databases store data.
=== Post 4 with dist=5.10: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 3 with dist=1.41


But...check out post 4

In [92]:
print(X_train.getrow(3).toarray())
print(X_train.getrow(4).toarray())

[[0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]]
[[0 0 0 0 3 3 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0]]
<class 'scipy.sparse.csr.csr_matrix'>


##Normalizing Word Count Vectors

In [101]:
def dist_norm(v1, v2):
    v1_normalized = v1/sp.linalg.norm(v1)
    v2_normalized = v2/sp.linalg.norm(v2)
    delta = v1_normalized - v2_normalized
    return sp.linalg.norm(delta)

find_euclidean_closest("imaging databases", dist_norm)

New post: imaging databases

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.92: Most imaging databases save images permanently.

=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 3 with dist=0.77


##Removing Less Important Words
**stop words** words like "most" that appear in all sorts of different contexts.

In [116]:
vectorizer = CountVectorizer(min_df=1, stop_words='english')
print(list(vectorizer.get_stop_words())[:20])
print(len(list(vectorizer.get_stop_words())))

['anyone', 'myself', 'cannot', 'my', 'off', 'via', 'also', 'un', 'even', 'sometime', 'whither', 'ever', 'between', 'wherein', 'everyone', 'some', 'someone', 'toward', 'ltd', 'through']
318


In [117]:
X_train = vectorizer.fit_transform(posts)
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d"%(num_samples, num_features))

#samples: 5, #features: 18


In [119]:
find_euclidean_closest("imaging databases", dist_norm, X_train)

New post: imaging databases

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.86: Most imaging databases save images permanently.

=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 3 with dist=0.77


Notice that post 2 is now on par with post 1, whereas before removing the stop words, it was up at .94

##Stemming
We currently counts similar words in different variants as different words.  E.g. "imaging" and "images."  **Stemming** reuces words to their specific word stem.

In [120]:
import nltk.stem

s = nltk.stem.SnowballStemmer('english')
print(s.stem('graphics'))
print(s.stem('imaging'))
print(s.stem('image'))
print(s.stem('imagination'))
print(s.stem('imagine'))
print(s.stem('buys'))
print(s.stem('buying'))
print(s.stem('bought'))

graphic
imag
imag
imagin
imagin
buy
buy
bought


###Extending the vectorizer with NLTK's stemmer


In [123]:
import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))
vectorizer = StemmedCountVectorizer(min_df=1, stop_words="english")

In [124]:
X_train = vectorizer.fit_transform(posts)
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d"%(num_samples, num_features))

#samples: 5, #features: 17


In [125]:
find_euclidean_closest("imaging databases", dist_norm, X_train)

New post: imaging databases

=== Post 0 with dist=1.41: This is a toy post about machine learning. Actually, it contains not much interesting stuff.
=== Post 1 with dist=0.86: Imaging databases provide storage capabilities.
=== Post 2 with dist=0.63: Most imaging databases save images permanently.

=== Post 3 with dist=0.77: Imaging databases store data.
=== Post 4 with dist=0.77: Imaging databases store data. Imaging databases store data. Imaging databases store data.
Best post is 2 with dist=0.63


Since the stemmed vectorizer collapses imaging and images, post 2 becomes the most similar because "imag" appears twice.

###Stop words on Steroids

*  `max_df` parameter can be set to fraction, e.g. .9, so that any word that occurs in more than 90 percent of of all posts will be ignored. But how low to set it?

* **Term frequency - inverse document frequency (TF_IDF)** - counting term frequencies for every post and discount those that appear in many posts.  We want a high value for a given term in a given post if that term occurs often in that post and very seldom anywhere else.


In [131]:
#Naive implementation
import scipy as sp
def tfidf(term, doc, corpus):
    #normalize so longer docs don't have advantage:
    tf = doc.count(term) / len(doc) 
    num_docs_with_term = len([d for d in corpus if term in d])
    idf = sp.log(len(corpus) / num_docs_with_term)
    return tf * idf

a, abb, abc = ['a'], ['a','b','b'], ['a','b','c']
D = [a, abb, abc]
print("a:")
print(tfidf("a", a, D))
print(tfidf("a", abb, D))
print(tfidf("a", abc, D))
print()
print(tfidf("b", abb, D))
print()
print(tfidf("b", abc, D))
print(tfidf("c", abc, D))

a:
0.0
0.0
0.0

0.270310072072

0.135155036036
0.366204096223


SciKit implements this with TfidfVectorizer, which is inherited from CountVectorizer

In [134]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')
class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (
            english_stemmer.stem(w) for w in analyzer(doc))
vectorizer = StemmedTfidfVectorizer(min_df=1,
                stop_words='english', decode_error='ignore')

In [135]:
X_train = vectorizer.fit_transform(posts)
num_samples, num_features = X_train.shape
print("#samples: %d, #features: %d"%(num_samples, num_features))

#samples: 5, #features: 17


###So far
1. Tokenize the text
    * lowercase the text
    * Throw away words that only occur once (min_df=1)
    * Throw away stop words
    * Collapse words onto their stems
2. Toss words that occur way too often to be of any help detcting related posts
3. Toss words that occur so seldom that they probably won't appear again.
4. Count Remaining words
5. Calculate TF-IDF values from counts, considering the whole text corpus

###Limits:
* **Does not cover word relations** "Car hits wall" and "Wall hits car" will have same feature vector
* **Does not capture negations**.  This can be fixed by also counting "bigrams" and "trigrams" in addition to "unigrams"
* **Cannot account for misspelled words**

#Clustering
* flat - i.e. key means.  Divides posts into a set of clusters without relating the clusters to each other.  Partitioning.  Needs up front number of clusters.
* hierarchical - clusters get clustered recursively.

In [143]:
import sklearn.datasets
all_data = sklearn.datasets.fetch_20newsgroups(subset='all')
print(len(all_data.filenames))
all_data.target_names

18846


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [142]:
groups = ['comp.graphics', 'comp.os.ms-windows.misc',
          'comp.sys.ibm.pc.hardware','comp.sys.mac.hardware',
          'comp.windows.x','sci.space']
train_data = sklearn.datasets.fetch_20newsgroups(
        subset='train', categories=groups)
print(len(train_data.filenames))

3529


In [146]:
test_data = sklearn.datasets.fetch_20newsgroups(
        subset='test', categories=groups)
print(len(test_data.filenames))

2349


In [149]:
vectorizer = StemmedTfidfVectorizer(
        min_df=10, max_df=0.5,
        stop_words='english',
        decode_error='ignore')
vectorized = vectorizer.fit_transform(train_data.data)
num_samples, num_features = vectorized.shape
print("#samples: %d, #features: %d"% (num_samples, num_features))

#samples: 3529, #features: 4712


In [150]:
num_clusters = 50
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters, init='random', 
            n_init=1, verbose=1, random_state=3)
km.fit(vectorized)

Initialization complete
Iteration  0, inertia 5899.560
Iteration  1, inertia 3218.298
Iteration  2, inertia 3184.333
Iteration  3, inertia 3164.867
Iteration  4, inertia 3152.004
Iteration  5, inertia 3143.111
Iteration  6, inertia 3136.256
Iteration  7, inertia 3129.325
Iteration  8, inertia 3124.567
Iteration  9, inertia 3121.900
Iteration 10, inertia 3120.210
Iteration 11, inertia 3118.627
Iteration 12, inertia 3117.363
Iteration 13, inertia 3116.811
Iteration 14, inertia 3116.588
Iteration 15, inertia 3116.417
Iteration 16, inertia 3115.760
Iteration 17, inertia 3115.374
Iteration 18, inertia 3115.155
Iteration 19, inertia 3114.949
Iteration 20, inertia 3114.515
Iteration 21, inertia 3113.937
Iteration 22, inertia 3113.720
Iteration 23, inertia 3113.548
Iteration 24, inertia 3113.475
Iteration 25, inertia 3113.447
Converged at iteration 25


KMeans(copy_x=True, init='random', max_iter=300, n_clusters=50, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=3, tol=0.0001,
    verbose=1)

In [151]:
# each cluster has been assigned a label
print(km.labels_)
print(km.labels_.shape)

[38 17 47 ..., 41 14 16]
(3529,)


##Putting it all together

In [170]:
new_post = """
Disk drive problems.  Hi, I have a problem with my hard disk.
After 1 year it is working only sporadically now.
I tried to format it, but now it doessn't boot any more.
Any ideas?  Thanks.
"""

new_post_vec = vectorizer.transform([new_post])
new_post_label = km.predict(new_post_vec)[0]

#Now that we have a clustering and a label...
# `nonzero` makes an array of indices of nonzero numbers
similar_indices = (km.labels_==new_post_label).nonzero()[0]

similar = []
for i in similar_indices:
    dist = sp.linalg.norm(
            new_post_vec - vectorized[i].toarray())
    similar.append((dist, train_data.data[i]))
similar = sorted(similar)
print(len(similar))

166


In [171]:
show_at_1 = similar[0]
show_at_2 = similar[int(len(similar)/10)]
show_at_3 = similar[int(len(similar)/ 2)]
show_at_1

(1.0236597431380716,
 "From: Thomas Dachsel <GERTHD@mvs.sas.com>\nSubject: BOOT PROBLEM with IDE controller\nNntp-Posting-Host: sdcmvs.mvs.sas.com\nOrganization: SAS Institute Inc.\nLines: 25\n\nHi,\nI've got a Multi I/O card (IDE controller + serial/parallel\ninterface) and two floppy drives (5 1/4, 3 1/2) and a\nQuantum ProDrive 80AT connected to it.\nI was able to format the hard disk, but I could not boot from\nit. I can boot from drive A: (which disk drive does not matter)\nbut if I remove the disk from drive A and press the reset switch,\nthe LED of drive A: continues to glow, and the hard disk is\nnot accessed at all.\nI guess this must be a problem of either the Multi I/o card\nor floppy disk drive settings (jumper configuration?)\nDoes someone have any hint what could be the reason for it.\nPlease reply by email to GERTHD@MVS.SAS.COM\nThanks,\nThomas\n+-------------------------------------------------------------------+\n| Thomas Dachsel                                        

##Another look at noise

In [175]:
post_group = zip(train_data.data, train_data.target)
all = [(len(post[0]), post[0], train_data.target_names[post[1]])
       for post in post_group]
graphics = sorted([post for post in all if 
                  post[2]=='comp.graphics'])
print(graphics[5])

(245, 'From: SITUNAYA@IBM3090.BHAM.AC.UK\nSubject: test....(sorry)\nOrganization: The University of Birmingham, United Kingdom\nLines: 1\nNNTP-Posting-Host: ibm3090.bham.ac.uk\n\n==============================================================================\n', 'comp.graphics')


In [176]:
noise_post = graphics[5][1]
analyzer = vectorizer.build_analyzer()
print(list(analyzer(noise_post)))

['situnaya', 'ibm3090', 'bham', 'ac', 'uk', 'subject', 'test', 'sorri', 'organ', 'univers', 'birmingham', 'unit', 'kingdom', 'line', 'nntp', 'post', 'host', 'ibm3090', 'bham', 'ac', 'uk']


In [177]:
useful = set(analyzer(noise_post)).intersection(
        vectorizer.get_feature_names())
print(sorted(useful))

['ac', 'birmingham', 'host', 'kingdom', 'nntp', 'sorri', 'test', 'uk', 'unit', 'univers']


In [178]:
#notice that all of these idf scores are pretty low.
#also, that the highest rated have nothing to do with 
#computer graphics
for term in sorted(useful):
    print('IDF(%s)=%.2f'%(term,
            vectorizer._tfidf.idf_[vectorizer.vocabulary_[term]]))

IDF(ac)=3.51
IDF(birmingham)=6.77
IDF(host)=1.74
IDF(kingdom)=6.68
IDF(nntp)=1.77
IDF(sorri)=4.14
IDF(test)=3.83
IDF(uk)=3.70
IDF(unit)=4.42
IDF(univers)=1.91


##Further Reading:
* Python 3 Text Processing with NLTK 3 Cookbook
* [sklearn clustering docs](http://scikit-learn.org/dev/modules/clustering.html)
* sklearn metrics docs - to learn more about measuring kmeans.